In [19]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 22.0 MB/s eta 0:00:00


In [20]:
!pip install PyPDF2 langchain langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.1 MB/s eta 0:00:00


In [21]:
!  pip install PyPDF2

In [22]:
!pip install  langchain langchain_community langchain_groq PyPDF2 concurrent

ERROR: Could not find a version that satisfies the requirement concurrent (from versions: none)
ERROR: No matching distribution found for concurrent


In [62]:
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_after_attempt, wait_exponential
import re
from langchain_groq import ChatGroq
import ast
import json


model_name = "llama3-8b-8192"
llm = ChatGroq(temperature=0, groq_api_key="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", model=model_name)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def split_text(text, chunk_size=30000, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return text_splitter.split_text(text)

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def extract_relations(chunk, llm):
    try:
        prompt = f"""
you are an expert indian paralegal who can analyse any type of legal file like Civil Suits, Writ Petitions, Appeals (Civil), Criminal Appeals, Criminal Revisions, Bail Applications, Public Interest Litigation (PIL), Company Petitions, Service Writs.

INTRO-
I have a legal case file, detect its type and I need you to do named entity recognition accordingly in a legal and professional manner for all possible entities.
and then a detailed relation extraction on all entities recognised to output tuples as (entity,relation,entity).

Identify the relationships between the extracted entity value and output a list of dictionaries containing head,relation,tail. Further head and tails will contain entity name and entity type each.
The tuples you generate will be used to develop a knowledge graph in accordance to the relations between entities later.

ACTUAL INPUT TEXT:
{chunk}

INSTRUCTIONS-
-you have to do all the extraction of tuples from the text mentioned above only which is the chunk
-no tuples should be repeated.
-revise the tuples before outputand only keep important and clear tuples which have valid values
-entities which are none,na or not defined should not be included.
-remove double quotes from the entities and relations in tuples. ("'court'" -> 'court')
-entity "type" and "relation" values should be uppercase

OUTPUT STRUCTURE-
{{"head": {{"name": entity1_name, "type": ENTITY1_TYPE}}, "relation": RELATION_NAME, "tail": {{"name": entity2_name, "type": ENTITY2_TYPE}}}},

THE BELOW EXAMPLE is NOT TO BE INCLUDED IN YOUR RESPONSE AS IT JUST FOR REFERENCE AND LEARNING
AN EXAMPLE OUTPUT DICTIONARY OF A RANDOM CRIMINAL FILE AND THE FORMAT OF THIS SHOULD BE IMITATED -
{{
  "head": {{"name": "High Court of Madhya Pradesh", "type": "COURT"}},
  "relation": "HAS_REGISTERED",
  "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "CASE_NUMBER"}}
}}

-the ouput you produce should strictly just be a list of dictionary of tuples and no other text or explanation should be generated

    """
        response = llm.predict(prompt)
        return response
    except Exception as e:
        print(f"Error processing chunk: {e}")
        raise

def process_chunks_batch(chunks, llm, batch_size=300):
    all_relations = []
    with ThreadPoolExecutor() as executor:
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i+batch_size]
            results = list(executor.map(lambda chunk: extract_relations(chunk, llm), batch))
            all_relations.extend(results)
    return all_relations

def combine_results(all_relations):
    return "\n".join(all_relations)

def extract_dictionaries(text):
    dictionaries = []
    current_dict = []
    in_dict = False
    bracket_depth = 0

    i = 0
    while i < len(text):
        char = text[i]

        if char == '{':
            bracket_depth += 1
            if bracket_depth == 1:
                current_dict = []
                in_dict = True
            current_dict.append(char)

        elif char == '}':
            current_dict.append(char)
            bracket_depth -= 1
            if bracket_depth == 0 and in_dict:
                in_dict = False
                try:
                    parsed_dict = ast.literal_eval("".join(current_dict))
                    if isinstance(parsed_dict, dict):
                        dictionaries.append(parsed_dict)
                except:
                    pass
                current_dict = []

        elif in_dict:
            current_dict.append(char)

        i += 1

    return dictionaries

#helper function for remove_duplicate_dicts
def dict_to_tuple(d):
    if isinstance(d, dict):
        return tuple((k, dict_to_tuple(v)) for k, v in sorted(d.items()))
    elif isinstance(d, list):
        return tuple(dict_to_tuple(v) for v in d)
    else:
        return d

def remove_duplicate_dicts(dict_list):
    seen = set()
    unique_dicts = []
    for d in dict_list:
        # Convert the dictionary to a tuple representation
        dict_tuple = dict_to_tuple(d)
        if dict_tuple not in seen:
            seen.add(dict_tuple)
            unique_dicts.append(d)
    return unique_dicts


@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def disambiguate_entities(tuples, llm):
    try:
        prompt = f"""
you are a legal expert who is very good at entity disambiguation.
You are provided with a dictionary of tuples representing entities and their relationships in the following format:
[
    "head": {{
        "name": "Entity Name",
        "type": "Entity Type"
    }},
    "relation": "Relationship Type",
    "tail": {{
        "name": "Entity Name",
        "type": "Entity Type"
    }}
]
Your task is to manually perform entity disambiguation on data by combining entities that are similar or have minor spelling differences. Additionally, ensure that the entity name and entity type are not the same (e.g., "name": "Government" and "type": "government" should be corrected).

Identify entities with similar names or minor spelling differences (e.g., "court", "Court", "court'", "COURT") and combine them into a single entity.
Correct any entities where the name and type are identical or very similar, ensuring that the type accurately represents the entity category.
Return the corrected dictionary with disambiguated entities and relationships.
Example Input:
[
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "COURT"}}, "relation": "HAS_REGISTERED", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "CASE_NUMBER"}}}},
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "COURT"}}, "relation": "DECIDED", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "CASE_NUMBER"}}}},
    {{"head": {{"name": "High Court of MP", "type": "COURT"}}, "relation": "REVIEWED", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "CASE_NUMBER"}}}},
    {{"head": {{"name": "court", "type": "COURT"}}, "relation": "HAS_CASE", "tail": {{"name": "case_number", "type": "CASE_NUMBER"}}}}
]

Expected Output:
[
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "COURT"}}, "relation": "HAS_REGISTERED", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "CASE_NUMBER"}}}},
    {{"head": {{"name": "High Court of Madhya Pradesh", "type": "COURT"}}, "relation": "DECIDED", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "CASE_NUMBER"}}}},
    {{"head": {{"name": "High Court of MP", "type": "COURT"}}, "relation": "REVIEWED", "tail": {{"name": "Cr. Appeal No. 262/2002", "type": "CASE_NUMBER"}}}},
    {{"head": {{"name": "court", "type": "COURT"}}, "relation": "HAS_CASE", "tail": {{"name": "case_number", "type": "CASE_NUMBER"}}}}
]

Actual input on which the processing has to be done- {tuples}
Instructions:
-entity "type" and "relation" values should be uppercase
-try to retain the size of tuples and dont lose entities
-Combine entities with minor spelling differences.
-Ensure entity names and types are not identical.
-Return the disambiguated and corrected dictionary in the given format
-your response should just contain the output data structure
        """
        response = llm.predict(prompt)
        return response
    except Exception as e:
        print(f"Error disambiguating entities: {e}")
        raise

def get_relation_tuples_from_pdf(pdf_path):
    content = extract_text_from_pdf(pdf_path)
    chunks = split_text(content)
    all_relations = process_chunks_batch(chunks, llm)
    final_result = combine_results(all_relations)
    dict_of_tuples = extract_dictionaries(final_result)
    #disambiguated_dict = disambiguate_entities(dict_of_tuples, llm)
    #extracted_dictionaries = extract_dictionaries(disambiguated_dict)
    #unique_extracted_dictionaries = remove_duplicate_dicts(extracted_dictionaries)
    return dict_of_tuples

def main(pdf_path):
    tuples=get_relation_tuples_from_pdf(pdf_path)
    return tuples

# Example usage:
pdf_path = '/content/Uday_vs_State_Of_Karnataka_on_19_February_2003.PDF'
response = main(pdf_path)

In [63]:
print(len(response),response)

24 [{'head': {'name': 'Uday', 'type': 'PERSON'}, 'relation': 'IS_ACCUSED', 'tail': {'name': 'State of Karnataka', 'type': 'STATE'}}, {'head': {'name': 'State of Karnataka', 'type': 'STATE'}, 'relation': 'FILED_CHARGES', 'tail': {'name': 'Uday', 'type': 'PERSON'}}, {'head': {'name': 'Bench', 'type': 'COURT'}, 'relation': 'HEARD_CASE', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}, 'relation': 'WAS_HEARD', 'tail': {'name': 'Bench', 'type': 'COURT'}}, {'head': {'name': 'Rao Harnarain Singh', 'type': 'PERSON'}, 'relation': 'DELIVERED_JUDGMENT', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Jayanti Rani Panda', 'type': 'PERSON'}, 'relation': 'CASE_REFERENCED', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Vijayan Pillai', 'type': 'PERSON'}, 'relation': 'CASE_REFERENCED', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}},

In [64]:
# Function to extract dictionaries from openyyai JSON file and concatenate them with the llm output dictionaries

def extract_and_concatenate(json_file_path, given_dicts):
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)

    extracted_dictionaries = json_data['tuples']  # Adjust this based on the actual structure of your JSON

    # Concatenate the dictionaries
    concatenated_dictionaries = given_dicts + extracted_dictionaries

    return concatenated_dictionaries

# Path to the uploaded JSON file
json_file_path = 'uday_openyyai_response.json'

# Extract and concatenate dictionaries
result = extract_and_concatenate(json_file_path, response)

print(len(result),result)

# If you want to save the result back to a file
with open('concatenated_dictionaries.json', 'w') as outfile:
    json.dump(result, outfile, indent=4)

129 [{'head': {'name': 'Uday', 'type': 'PERSON'}, 'relation': 'IS_ACCUSED', 'tail': {'name': 'State of Karnataka', 'type': 'STATE'}}, {'head': {'name': 'State of Karnataka', 'type': 'STATE'}, 'relation': 'FILED_CHARGES', 'tail': {'name': 'Uday', 'type': 'PERSON'}}, {'head': {'name': 'Bench', 'type': 'COURT'}, 'relation': 'HEARD_CASE', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}, 'relation': 'WAS_HEARD', 'tail': {'name': 'Bench', 'type': 'COURT'}}, {'head': {'name': 'Rao Harnarain Singh', 'type': 'PERSON'}, 'relation': 'DELIVERED_JUDGMENT', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Jayanti Rani Panda', 'type': 'PERSON'}, 'relation': 'CASE_REFERENCED', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Vijayan Pillai', 'type': 'PERSON'}, 'relation': 'CASE_REFERENCED', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}

In [65]:
#handling duplicates after concat
result=remove_duplicate_dicts(result)
print(len(result),result)


105 [{'head': {'name': 'Uday', 'type': 'PERSON'}, 'relation': 'IS_ACCUSED', 'tail': {'name': 'State of Karnataka', 'type': 'STATE'}}, {'head': {'name': 'State of Karnataka', 'type': 'STATE'}, 'relation': 'FILED_CHARGES', 'tail': {'name': 'Uday', 'type': 'PERSON'}}, {'head': {'name': 'Bench', 'type': 'COURT'}, 'relation': 'HEARD_CASE', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}, 'relation': 'WAS_HEARD', 'tail': {'name': 'Bench', 'type': 'COURT'}}, {'head': {'name': 'Rao Harnarain Singh', 'type': 'PERSON'}, 'relation': 'DELIVERED_JUDGMENT', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Jayanti Rani Panda', 'type': 'PERSON'}, 'relation': 'CASE_REFERENCED', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}, {'head': {'name': 'Vijayan Pillai', 'type': 'PERSON'}, 'relation': 'CASE_REFERENCED', 'tail': {'name': 'Uday vs State Of Karnataka', 'type': 'CASE'}}

In [31]:
!pip install neo4j


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 6.4 MB/s eta 0:00:00


In [32]:
from neo4j import GraphDatabase

# Replace these with your AuraDB credentials
uri = "neo4j+s://6cb5ec24.databases.neo4j.io"
user = "neo4j"
password = "Ww3XWwg1qPUkKzeCoxmBGeM0Jw9pY3HZLQ2V7vzQwjA"



In [33]:
pip install py2neo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 4.1 MB/s eta 0:00:00


In [34]:
pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.5 MB/s eta 0:00:00


In [66]:
from neo4j import GraphDatabase
import logging

# Create a connection to AuraDB
def create_auradb_connection(uri, user, password):
    try:
        driver = GraphDatabase.driver(uri, auth=(user, password))
        return driver
    except Exception as e:
        logging.error(f"Error creating AuraDB connection: {e}")
        raise

# Function to insert a tuple into AuraDB
def insert_tuple(tx, entity1_value, entity1_type, relation, entity2_value, entity2_type):
    try:
        query = f"""
                    MERGE (a:{entity1_type} {{name: $entity1_value}})
                    MERGE (b:{entity2_type} {{name: $entity2_value}})
                    MERGE (a)-[r:{relation}]->(b)
                """
        tx.run(query, entity1_value=entity1_value, entity1_type=entity1_type,
               relation=relation, entity2_value=entity2_value, entity2_type=entity2_type)
    except Exception as e:
        logging.error(f"Error inserting tuple: {e}")
        raise

# Process and insert dictionaries into AuraDB
def process_and_insert_dicts(uri, user, password, dicts):
    driver = create_auradb_connection(uri, user, password)
    try:
        with driver.session() as session:
            for d in dicts:
                head = d['head']
                tail = d['tail']
                try:
                    session.write_transaction(
                        insert_tuple,
                        head['name'], head['type'],
                        d['relation'],
                        tail['name'], tail['type']
                    )
                except Exception as e:
                    logging.error(f"Error processing dictionary: {d}, Error: {e}")
                    continue
    except Exception as e:
        logging.error(f"Error during session operation: {e}")
        raise
    finally:
        driver.close()

# Process and insert dictionaries into AuraDB
process_and_insert_dicts(uri, user, password, result)


<ipython-input-66-fe599f831fde>:36: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(


In [67]:
#RENDERING EXISTING KNOWLEDGE GRAPH FROM AURADB AS HTML(WITH COLOUR)
from py2neo import Graph
from pyvis.network import Network
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random


driver = GraphDatabase.driver(uri, auth=(user, password))
graph = Graph(uri, auth=(user, password))

# Define a Cypher query to retrieve nodes and relationships
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 400
"""

result = graph.run(query)

# Create a pyvis network
net = Network(height='1000px', width='100%', notebook=True, bgcolor='#FAF9F6', font_color='black')

# Function to generate a random color
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Dictionary to store colors for each label
label_colors = {}

# Add nodes and edges from the query result
for record in result:
    n = record["n"]
    r = record["r"]
    m = record["m"]

    # Ensure names and labels are strings for serialization
    n_name = str(n["name"])
    m_name = str(m["name"])
    n_labels = list(n.labels)
    m_labels = list(m.labels)
    r_type = r.__class__.__name__
    # Assign colors to labels dynamically
    if n_labels:
        for label in n_labels:
            if label not in label_colors:
                label_colors[label] = random_color()
        n_color = label_colors[n_labels[0]]
    else:
        n_color = random_color()

    if m_labels:
        for label in m_labels:
            if label not in label_colors:
                label_colors[label] = random_color()
        m_color = label_colors[m_labels[0]]
    else:
        m_color = random_color()

    net.add_node(n_name, label=n_name, title=", ".join(n_labels), color=n_color)
    net.add_node(m_name, label=m_name, title=", ".join(m_labels), color=m_color)
    net.add_edge(n_name, m_name,title=r_type, color='#FFD700')

# Generate network with options
net.show_buttons(filter_=['physics'])
net.show('knowledge_graph.html')


knowledge_graph.html
